In [4]:
import sys, os
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
#import weatherbench2 as wb
#import metpy
import copy
import wrf
import time
#from metpy.units import units
import cartopy.crs as ccrs
print('modules loaded')
import datetime
import glob
import datetime as dt
import seaborn as sns
import netCDF4 as nc
import metpy
from metpy.units import units
from glob import glob

modules loaded


In [73]:
figpath='/users/mfeldman/figs/case_24/'
datapath='/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/'
models=['aifs','pangu','graphcast','tigge']

In [74]:
era5='gs://weatherbench2/datasets/era5/1959-2022-6h-1440x721.zarr'
xr_era5=xr.open_zarr(era5).sortby('latitude')
xr_era5=xr_era5.sel(time=xr_era5.time.dt.year.isin([2020]))
xr_era5=xr_era5.sel(time=xr_era5.time.dt.month.isin([4]))#np.arange(3,10)))
xr_era5=xr_era5.sel(time=xr_era5.time.dt.day.isin([2,3]),latitude=slice(20,50),longitude=slice(250,300))
zsurf_c=xr_era5['geopotential_at_surface']

year=2024; month=4; day=np.arange(2,4,1); days=day[0]-np.arange(1,10)
print(days)
latslice2=slice(20,50)
for model in models[-1:]:
    files_pl=sorted(glob(datapath+model+'/pl*.nc'))
    files_sfc=sorted(glob(datapath+model+'/sfc*.nc'))
    for nn in range(len(files_pl))[:]:
        st=datetime.datetime(2024,4,2,0)
        et=datetime.datetime(2024,4,3,23)


        print(files_pl[nn])
        xr_pl=xr.open_dataset(files_pl[nn]).sortby('latitude')
        init=np.datetime_as_string(xr_pl.time[0].values,'h')
        
        xr_pl=xr_pl.sel(time=xr_pl.time.dt.year.isin([year]))
        xr_pl=xr_pl.sel(time=xr_pl.time.dt.month.isin([month]))
        xr_pl=xr_pl.sel(time=xr_pl.time.dt.day.isin([day]))
        
        xr_sfc=xr.open_dataset(files_sfc[nn]).sortby('latitude')
        xr_sfc=xr_sfc.sel(time=xr_sfc.time.dt.year.isin([year]))
        xr_sfc=xr_sfc.sel(time=xr_sfc.time.dt.month.isin([month]))
        xr_sfc=xr_sfc.sel(time=xr_sfc.time.dt.day.isin([day]))


        xr_pl = xr_pl.sortby('level', ascending=False)
        
        plevel=copy.deepcopy(xr_pl.level.values)
        
        tlevel=xr_pl.t
        qlevel=xr_pl.q
        plevel_dim=np.ones([1,len(plevel),1,1])
        if model=='tigge': zlevel=xr_pl.gh
        else: zlevel=xr_pl.z
        ulevel=xr_pl.u
        vlevel=xr_pl.v
        for n1 in range(len(plevel.data)):
            plevel_dim[0,n1,0,0]=plevel[n1]
        plevel_exp=(tlevel/tlevel)*plevel_dim
        
        psurf=xr_sfc.msl
        tsurf=xr_sfc.t2m
        zsurf = zsurf_c.expand_dims(dim={"time": psurf.time}, axis=0)
        # zsurf = zsurf.expand_dims(dim={"time": psurf.time}, axis=1)
        
        ps=psurf/100
        zs=zsurf/9.81
        zl=zlevel/9.81

        mod_inst=wrf.cape_2d(pres_hpa=plevel_exp, tkel=tlevel, qv=qlevel, height=zl, terrain=zs, psfc_hpa=psurf, ter_follow=False)
        mod_inst=mod_inst.assign_coords(longitude=psurf.longitude.values);
        mod_inst=mod_inst.assign_coords(latitude=psurf.latitude.values);
        mod_inst=mod_inst.assign_coords(time=psurf.time.values)

        du_01=ulevel.sel(level=850)-xr_sfc.u10; dv_01=vlevel.sel(level=850)-xr_sfc.v10
        du_03=ulevel.sel(level=700)-xr_sfc.u10; dv_03=vlevel.sel(level=700)-xr_sfc.v10
        du_06=ulevel.sel(level=500)-xr_sfc.u10; dv_06=vlevel.sel(level=500)-xr_sfc.v10
        
        bs_01=( du_01**2 + dv_01**2 )**0.5
        bs_03=( du_03**2 + dv_03**2 )**0.5
        bs_06=( du_06**2 + dv_06**2 )**0.5
        
        du_01=du_01.to_dataset(name='du_01')
        dv_01=dv_01.to_dataset(name='dv_01')
        bs_01=bs_01.to_dataset(name='bs_01')
        du_03=du_03.to_dataset(name='du_03')
        dv_03=dv_03.to_dataset(name='dv_03')
        bs_03=bs_03.to_dataset(name='bs_03')
        du_06=du_06.to_dataset(name='du_06')
        dv_06=dv_06.to_dataset(name='dv_06')
        bs_06=bs_06.to_dataset(name='bs_06')
        
        mod_params=xr.merge([du_01,dv_01,bs_01,du_03,dv_03,bs_03,du_06,dv_06,bs_06],compat='override')
        #mod_params=mod_params.squeeze(dim='level')
        mod_params['cape']=mod_inst.sel(mcape_mcin_lcl_lfc='mcape')
        mod_params['cin']=mod_inst.sel(mcape_mcin_lcl_lfc='mcin')
        mod_params['lcl']=mod_inst.sel(mcape_mcin_lcl_lfc='lcl')
        mod_params['lfc']=mod_inst.sel(mcape_mcin_lcl_lfc='lfc')#.squeeze(dim='mcape_mcin_lcl_lfc')
        print(model,init)
        mod_params.to_netcdf(datapath+'postproc/'+model+'_'+init+'.nc')

        cm=plt.cm.Reds
        cm.set_under(color="grey")
        cm.set_bad(color="grey")
        p = mod_params.cape.sel(latitude=slice(20,40),longitude=slice(260,290)).plot(x="longitude", y="latitude", col="time", col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()}\
                        ,cmap=cm,vmin=200,vmax=1500,levels=np.arange(200,1500,100),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+'_'+init+'_cape.png',  dpi=300, bbox_inches='tight')
        plt.close()
        
        cm=plt.cm.Reds
        cm.set_under(color="grey")
        cm.set_bad(color="grey")
        p = mod_params.bs_06.sel(latitude=slice(20,40),longitude=slice(260,290)).plot(x="longitude", y="latitude", col="time", col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()}\
                        ,cmap=cm,vmin=5,vmax=30,levels=np.arange(5,32,2),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+'_'+init+'_bs_06.png',  dpi=300, bbox_inches='tight')
        plt.close()


[ 1  0 -1 -2 -3 -4 -5 -6 -7]
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-23_00:00:00.nc
tigge 2024-03-23T00
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-23_12:00:00.nc
tigge 2024-03-23T12
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-24_00:00:00.nc
tigge 2024-03-24T00
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-24_12:00:00.nc
tigge 2024-03-24T12
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-25_00:00:00.nc
tigge 2024-03-25T00
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-25_12:00:00.nc
tigge 2024-03-25T12
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-26_00:00:00.nc
tigge 2024-03-26T00
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-26_12:00:00.nc
tigge 2024-03-26T12
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-27_00:00:00.nc
tigge 2024-03-27T00

In [75]:
for model in models[-1:]:
    files_pl=sorted(glob(datapath+model+'/pl*.nc'))
    files_sfc=sorted(glob(datapath+model+'/sfc*.nc'))
    for nn in range(len(files_pl))[:]:
        st=datetime.datetime(2024,4,2,0)
        et=datetime.datetime(2024,4,3,23)


        print(files_pl[nn])
        xr_pl=xr.open_dataset(files_pl[nn]).sortby('latitude')
        init=np.datetime_as_string(xr_pl.time[0].values,'h')
        
        xr_sfc=xr.open_dataset(files_sfc[nn]).sortby('latitude')
        xr_sfc=xr_sfc.sel(time=xr_sfc.time.dt.year.isin([year]))
        xr_sfc=xr_sfc.sel(time=xr_sfc.time.dt.month.isin([month]))
        xr_sfc=xr_sfc.sel(time=xr_sfc.time.dt.day.isin([day]))

        cm=plt.cm.Reds
        cm.set_under(color="grey")
        cm.set_bad(color="grey")
        p = xr_sfc.cape.sel(latitude=slice(20,40),longitude=slice(260,290)).plot(x="longitude", y="latitude", col="time", col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()}\
                        ,cmap=cm,vmin=200,vmax=1500,levels=np.arange(200,1500,100),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+'_'+init+'_cape_mod.png',  dpi=300, bbox_inches='tight')
        plt.close()

/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-23_00:00:00.nc
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-23_12:00:00.nc
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-24_00:00:00.nc
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-24_12:00:00.nc
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-25_00:00:00.nc
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-25_12:00:00.nc
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-26_00:00:00.nc
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-26_12:00:00.nc
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-27_00:00:00.nc
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-27_12:00:00.nc
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/tigge/pl_2024-03-28_00:00:00.nc
/work/FAC/FGSE/IDYST/tbeucler/de

In [71]:
models2=['sfno','fuxi']
year=2024; month=4; day=np.arange(2,4,1); days=day[0]-np.arange(1,10)
print(days)
latslice2=slice(20,50)
for model in models2[:]:
    files_pl=sorted(glob(datapath+model+'/pl*.nc'))
    files_sfc=sorted(glob(datapath+model+'/sfc*.nc'))
    for nn in range(len(files_pl))[:]:
        st=datetime.datetime(2024,4,2,0)
        et=datetime.datetime(2024,4,3,23)


        print(files_pl[nn])
        xr_pl=xr.open_dataset(files_pl[nn]).sortby('latitude')
        init=np.datetime_as_string(xr_pl.time[0].values,'h')
        
        xr_pl=xr_pl.sel(time=xr_pl.time.dt.year.isin([year]))
        xr_pl=xr_pl.sel(time=xr_pl.time.dt.month.isin([month]))
        xr_pl=xr_pl.sel(time=xr_pl.time.dt.day.isin([day]))
        
        xr_sfc=xr.open_dataset(files_sfc[nn]).sortby('latitude')
        xr_sfc=xr_sfc.sel(time=xr_sfc.time.dt.year.isin([year]))
        xr_sfc=xr_sfc.sel(time=xr_sfc.time.dt.month.isin([month]))
        xr_sfc=xr_sfc.sel(time=xr_sfc.time.dt.day.isin([day]))


        xr_pl = xr_pl.sortby('level', ascending=False)
        
        plevel=copy.deepcopy(xr_pl.level.values)
        
        tlevel=xr_pl.t
        rlevel=xr_pl.r
        plevel_dim=np.ones([1,len(plevel),1,1])
        zlevel=xr_pl.z
        ulevel=xr_pl.u
        vlevel=xr_pl.v
        for n1 in range(len(plevel.data)):
            plevel_dim[0,n1,0,0]=plevel[n1]
        plevel_exp=(tlevel/tlevel)*plevel_dim

        dplevel=metpy.calc.dewpoint_from_relative_humidity(tlevel.values* units('K'), rlevel.values* units.percent)
        qlevel=metpy.calc.specific_humidity_from_dewpoint(plevel_exp.values* units('hPa'), dplevel).magnitude
        
        psurf=xr_sfc.msl
        tsurf=xr_sfc.t2m
        zsurf = zsurf_c.expand_dims(dim={"time": psurf.time}, axis=0)
        # zsurf = zsurf.expand_dims(dim={"time": psurf.time}, axis=1)
        
        ps=psurf/100
        zs=zsurf/9.81
        zl=zlevel/9.81

        mod_inst=wrf.cape_2d(pres_hpa=plevel_exp, tkel=tlevel, qv=qlevel, height=zl, terrain=zs, psfc_hpa=psurf, ter_follow=False)
        mod_inst=mod_inst.assign_coords(longitude=psurf.longitude.values);
        mod_inst=mod_inst.assign_coords(latitude=psurf.latitude.values);
        mod_inst=mod_inst.assign_coords(time=psurf.time.values)

        du_01=ulevel.sel(level=850)-xr_sfc.u10; dv_01=vlevel.sel(level=850)-xr_sfc.v10
        du_03=ulevel.sel(level=700)-xr_sfc.u10; dv_03=vlevel.sel(level=700)-xr_sfc.v10
        du_06=ulevel.sel(level=500)-xr_sfc.u10; dv_06=vlevel.sel(level=500)-xr_sfc.v10
        
        bs_01=( du_01**2 + dv_01**2 )**0.5
        bs_03=( du_03**2 + dv_03**2 )**0.5
        bs_06=( du_06**2 + dv_06**2 )**0.5
        
        du_01=du_01.to_dataset(name='du_01')
        dv_01=dv_01.to_dataset(name='dv_01')
        bs_01=bs_01.to_dataset(name='bs_01')
        du_03=du_03.to_dataset(name='du_03')
        dv_03=dv_03.to_dataset(name='dv_03')
        bs_03=bs_03.to_dataset(name='bs_03')
        du_06=du_06.to_dataset(name='du_06')
        dv_06=dv_06.to_dataset(name='dv_06')
        bs_06=bs_06.to_dataset(name='bs_06')
        
        mod_params=xr.merge([du_01,dv_01,bs_01,du_03,dv_03,bs_03,du_06,dv_06,bs_06],compat='override')
        #mod_params=mod_params.squeeze(dim='level')
        mod_params['cape']=mod_inst.sel(mcape_mcin_lcl_lfc='mcape')
        mod_params['cin']=mod_inst.sel(mcape_mcin_lcl_lfc='mcin')
        mod_params['lcl']=mod_inst.sel(mcape_mcin_lcl_lfc='lcl')
        mod_params['lfc']=mod_inst.sel(mcape_mcin_lcl_lfc='lfc')#.squeeze(dim='mcape_mcin_lcl_lfc')
        print(model,init)
        mod_params.to_netcdf(datapath+'postproc/'+model+'_'+init+'.nc')

        cm=plt.cm.Reds
        cm.set_under(color="grey")
        cm.set_bad(color="grey")
        p = mod_params.cape.sel(latitude=slice(20,40),longitude=slice(260,290)).plot(x="longitude", y="latitude", col="time", col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()}\
                        ,cmap=cm,vmin=200,vmax=1500,levels=np.arange(200,1500,100),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+'_'+init+'_cape.png',  dpi=300, bbox_inches='tight')
        plt.close()
        
        cm=plt.cm.Reds
        cm.set_under(color="grey")
        cm.set_bad(color="grey")
        p = mod_params.bs_06.sel(latitude=slice(20,40),longitude=slice(260,290)).plot(x="longitude", y="latitude", col="time", col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()}\
                        ,cmap=cm,vmin=5,vmax=30,levels=np.arange(5,32,2),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+'_'+init+'_bs_06.png',  dpi=300, bbox_inches='tight')
        plt.close()
        



[ 1  0 -1 -2 -3 -4 -5 -6 -7]
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/sfno/pl_2024-03-23_00:00:00.nc


/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/metpy/calc/thermo.py:1396: RuntimeWarning: invalid value encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)


sfno 2024-03-23T00
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/sfno/pl_2024-03-23_12:00:00.nc


/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/metpy/calc/thermo.py:1396: RuntimeWarning: invalid value encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)


sfno 2024-03-23T12
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/sfno/pl_2024-03-24_00:00:00.nc


...

fuxi 2024-04-02T12
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/fuxi/pl_2024-04-03_00:00:00.nc


/tmp/ipykernel_2661966/2212193301.py:41: UserWarning: Relative humidity >120%, ensure proper units.
  dplevel=metpy.calc.dewpoint_from_relative_humidity(tlevel.values* units('K'), rlevel.values* units.percent)
/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/metpy/calc/thermo.py:1396: RuntimeWarning: invalid value encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)


fuxi 2024-04-03T00
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF_AI/fuxi/pl_2024-04-03_12:00:00.nc


/tmp/ipykernel_2661966/2212193301.py:41: UserWarning: Relative humidity >120%, ensure proper units.
  dplevel=metpy.calc.dewpoint_from_relative_humidity(tlevel.values* units('K'), rlevel.values* units.percent)
/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/metpy/calc/thermo.py:1396: RuntimeWarning: invalid value encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)


fuxi 2024-04-03T12
